# B92 Protocol

## Protocol Steps
1. Alice generates a random bit string and creates a corresponding qubit string that is mapped from the bit string as follows: 0 -> |z+>, 1 -> |x+>. Alice then sends these qubits to Bob over a public channel
2. Bob generates a random bit string and measures Alice's qubits with the following mapping from his bit string: 0 -> {|x+>, |x->}, 1: -> {|z+>, |z->}
3. Bob sends his measured bits back to Alice over a public channel.
4. Alice and Bob both remoove all the bits in their strings corresponding to 0s in the string bob just sent

### Import packages

In [23]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd

### Define all the meta parameters we will need for the protocol
- n: the number of bits that Alice and Bob will randomly generate in the protocol
- test: boolean that will standardize the pseudo-random fucntions in the protocol and print the steps of the protocol along the way

In [24]:
n = 30
test = True

if test:
    np.random.seed(seed=0)

### Create the qubits that Alice will send across the public channel

In [25]:
def alice_encode(input_arr):
    output = []
    for bit in input_arr:
        qc = QuantumCircuit(1,1)
        
        if bit == 1:
            qc.h(0)
        qc.barrier()
        output.append(qc)
    return output

In [26]:
alice_bits = randint(2, size=n)
alice_qubits = alice_encode(alice_bits)


if test:
    
    print('Alice bits:')
    print(alice_bits)
    
#     print('Alice qubits:')
#     for i in alice_qubits:
#         print(i)


Alice bits:
[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0]


### Bob measures the qubits and sends a bit string back to Alice across the public channel
- We define a function that take a list of qubits and a list of bits. We loop through the lists in parallel and if the corresponding bit is a 0, we measure the qubit in the x-basis and in the z-basis otherwise. We return all of the outputs in a list

In [27]:
def bob_decode(alice_qubits, bob_bits):
    output = []
    for alice_qubit, bob_bit in zip(alice_qubits, bob_bits):
        
        # Measure in the x-basis
        if bob_bit == 0:
            alice_qubit.h(0)
            alice_qubit.measure(0, 0)
        
        # Measure in the z-basis
        else:
            alice_qubit.measure(0, 0)
            
        # Simulate the measurement (taken directly from the qiskit example)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(alice_qubit, shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        output.append(measured_bit)
    return output    

In [28]:
bob_bits = randint(2, size=n)
bob_yield = bob_decode(alice_qubits, bob_bits)

if test:
    print('Bob bits:')
    print(bob_bits)
    
    print('Bob yeild')
    print(bob_yield)


Bob bits:
[1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1]
Bob yeild
[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]


### Create the shared keys
- We define a function that takes both Alice and Bobs original strings and removes all of the bits that correspond to a 0 in the bits Bob measured. We return a tuple of stirngs that are a concatenation of these bits

In [29]:
def remove_zeros(alice_bits, bob_bits, bob_yield):
    bob_key = ''
    alice_key = ''
    
    for a_bit, b_bit, b_yield in zip(alice_bits, bob_bits, bob_yield):
        if b_yield == 1:
            bob_key += str(b_bit)
            alice_key += str(a_bit)

    return alice_key, bob_key

In [30]:
alice_key, bob_key = remove_zeros(alice_bits, bob_bits, bob_yield)

if test:
    print('Key stirings')
    print(f'Alice key: {alice_key}')
    print(f'Bob key: {bob_key}')

Key stirings
Alice key: 1111101
Bob key: 1111101


### Testing Accuracy
- We can test if the protocol worked and Alice and Bob have the same keys

In [31]:
if bob_key == alice_key:
    print('The protocol worked and Alice and Bob have the same keys!')
else:
    print('Something went wrong! Alice and Bob do not have the same keys!')

The protocol worked and Alice and Bob have the same keys!
